This project prompts the user to give their thoughts on a movie they have seen. Then, it will provide an estimation of what that person would rate the movie out of 5, with 5 being the best and 1 being the worst.

In [187]:
# Imports metric dataframe if already calculated
import pandas as pd
df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')
import math

In [188]:
reviews = []
ratings = []
data = []
for review in df.itertuples(index=False):
    if (type(review[5]) == str and type(review[7]) == str):
        reviews.append(review[7])
        if review[5] == 'A+':
            ratings.append('5/5')
        elif review[5] == 'A':
            ratings.append('4.5/5')
        elif review[5] == 'A-':
            ratings.append('4.75/5')
        elif review[5] == 'A  -':
            ratings.append('4.75/5')
        elif review[5] == 'B+':
            ratings.append('4/5')
        elif review[5] == 'B':
            ratings.append('3.5/5')
        elif review[5] == 'B-':
            ratings.append('3.25/5')
        elif review[5] == 'C+':
            ratings.append('3/5')
        elif review[5] == 'C':
            ratings.append('2.5/5')
        elif review[5] == 'C-':
            ratings.append('2.25/5')
        elif review[5] == 'C  -':
            ratings.append('2.25/5')
        elif review[5] == 'D+':
            ratings.append('2/5')
        elif review[5] == 'D':
            ratings.append('1.5/5')
        elif review[5] == 'D-':
            ratings.append('1.25/5')
        elif review[5] == 'F+':
            ratings.append('1/5')
        elif review[5] == 'F':
            ratings.append('0.5/5')
        elif review[5] == 'F-':
            ratings.append('0.25/5')
        else:
            ratings.append(review[5])

In [189]:
ratings_as_float= []
for i in ratings:
    if '/' in i:
        a , b = i.split("/")
        if float(b) == 0:
            b = 100
        to_float = float(a) / float(b)
        ratings_as_float.append(to_float)
    else:
        if float(i) < 10:
            ratings_as_float.append(float(i)/10)
        elif float(i) <= 100:
            ratings_as_float.append(float(i)/100)
        else:
            ratings_as_float.append(float(i)/1000)

In [190]:
Data = pd.DataFrame({'Review':reviews,'Rating':ratings_as_float})

In [ ]:
import nltk

,Review,Rating
0,Whether audiences will get behind The Lightnin...,0.70
1,Harry Potter knockoffs don't come more transpa...,0.25
2,"Percy Jackson isn't a great movie, but it's a ...",0.70
3,"Fun, brisk and imaginative",0.70
4,"Crammed with dragons, set-destroying fights an...",0.60
...,...,...
758704,As a spectacular war film with a powerful mora...,0.80
758705,"Seen today, it's not only a startling indictme...",0.70
758706,A rousing visual spectacle that's a prequel of...,0.80
758707,"A simple two-act story: Prelude to war, and th...",0.70


In [ ]:
tokens = []
for review in Data['Review']:
    token = nltk.word_tokenize(review)
    tokens.append(token)

In [ ]:
import fasttext

In [ ]:
a